In [1]:
import pandas as pd
import os,sys,inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)

train_1 = pd.read_csv(parentdir + "/Dataset/sub_navigate_1.csv", encoding="utf-8")
train_2 = pd.read_csv(parentdir + "/Dataset/sub_navigate_2.csv", encoding="utf-8")

In [2]:
train_1.head()
train_2.head()

,utterance
0,4마일 이내의 내 친구네 집까지 가는 길은 어디입니까?
1,5672 바링거 가까지 가는 길을 알려 주세요.
2,5677 남서쪽 4번가로 가는 길을 가르쳐 주세요.
3,5마일 이내의 병원과 가장 빠른 길로 가주세요.
4,가까운 곳으로 가는 길을 좀 가르쳐 주세요. 커피를 마실 수 있어요.


In [3]:
print(len(train_1["utterance"]),len(train_2["utterance"]))

358 233


In [4]:
trn = pd.DataFrame(train_1["utterance"])
trn["label"]= 1
trn.head()

trw = pd.DataFrame(train_2["utterance"])
trw["label"]= 2
trw.head()

,utterance,label
0,4마일 이내의 내 친구네 집까지 가는 길은 어디입니까?,2
1,5672 바링거 가까지 가는 길을 알려 주세요.,2
2,5677 남서쪽 4번가로 가는 길을 가르쳐 주세요.,2
3,5마일 이내의 병원과 가장 빠른 길로 가주세요.,2
4,가까운 곳으로 가는 길을 좀 가르쳐 주세요. 커피를 마실 수 있어요.,2


In [5]:
data = trn.append(trw)

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

In [7]:
# 전체 데이터를 train : test = 8:2 로 분할
train, test = train_test_split(data, test_size=0.20, random_state=42)

In [8]:
train["label"] = pd.Categorical(train["label"])
test["label"] = pd.Categorical(test["label"])

/home/judy/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/judy/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [9]:
train.groupby("label").count()
test.groupby("label").count()

,utterance
label,
1,76
2,43


In [10]:
from konlpy.tag import Okt

In [11]:
from konlpy.tag import Okt
def get_noun(text):
    tokenizer = Okt()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

In [12]:
from konlpy.tag import Okt
def get_stem(text):
    tokenizer = Okt()
    stems = tokenizer.morphs(text)
    return [n for n in stems]

Solution : Pipeline
---

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer(max_features = 100, max_df=0.95, min_df=0)
text_clf_svm = Pipeline([('vect', TfidfVectorizer(tokenizer=get_stem, max_features = 100, max_df=0.95, min_df=0)),
                          ('tfidf', TfidfTransformer()), ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=1000, random_state=42))])

text_clf_svm = text_clf_svm.fit(train["utterance"], train["label"])

from sklearn.externals import joblib
joblib.dump(text_clf_svm, 'model_navigate.joblib') 

/home/judy/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


['model_navigate.joblib']

In [14]:
predicted_svm = text_clf_svm.predict(test["utterance"])
np.mean(predicted_svm == test["label"])

0.9327731092436975

In [15]:
def sub_intent(str):
    intent = text_clf_svm.predict([str])
    
    if intent == 1:
        print("sub intent : 주소/위치 찾기")
    else:
        print("sub intent : 경로 찾기")

1. 주소, 위치 찾기

In [26]:
sub_intent("근처에 편의점 찾아줘")

sub intent : 주소/위치 찾기


In [27]:
sub_intent("국민대학교 주소 알려줘")

sub intent : 주소/위치 찾기


In [28]:
sub_intent("가장 가까운 스타벅스를 찾습니다.")

sub intent : 주소/위치 찾기


2. 경로 찾기

In [29]:
sub_intent("국민대학교 가는 길 알려줘")

sub intent : 경로 찾기


In [30]:
sub_intent("집에 가는 길 찾아줘")

sub intent : 경로 찾기


In [31]:
sub_intent("집으로 가는 가장 빠른 경로를 탐색합니다.")

sub intent : 경로 찾기
